In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder.appName('SparkStreaming').getOrCreate()
spark.sparkContext.setLogLevel("WARN")
spotify_schema = spark.read.format('json').load('file:////home/swap/Documents/Task9_Spotify/spotify_sample.txt').schema

Py4JError: org.apache.spark.api.python.PythonUtils.isEncryptionEnabled does not exist in the JVM

In [ ]:
df = spark.readStream.format('kafka').option('kafka.bootstrap.servers','localhost:9093').option('subscribe','spotify').option('startingOffsets','earliest').load()

In [ ]:
df_data = df.select(from_json(col("value").cast("string"),spotify_schema).alias('parsed_value')).selectExpr('parsed_value.*')

In [ ]:
df_final = df_data.withColumn('album_name',col('album.name')).withColumn('release_date',col('album.release_date')).selectExpr('name as track_name','artist[0].name as artist','album_name','release_date','popularity','id as track_id')

In [ ]:
out = df_final.writeStream.format('console').outputMode('append').start()

In [ ]:
out.awaitTermination()